In [30]:
import matplotlib
import mxnet as mx
import numpy as np
from mxnet import gluon, nd
from mxnet.gluon.data.vision import transforms
from gluoncv.data import imagenet
from collections import namedtuple
import multiprocessing
from mxnet.contrib.onnx.onnx2mx.import_model import import_model

In [33]:
# Determine and set context
if len(mx.test_utils.list_gpus())==0:
    ctx = [mx.cpu()]
else:
    ctx = [mx.gpu(0)]

# path to imagenet dataset folder
data_dir = '/home/jiang784/Benchmark/dataset/imagenet/val'

# batch size (set to 1 for cpu)
batch_size = 128

# number of preprocessing workers
num_workers = multiprocessing.cpu_count()

# path to ONNX model file
model_path = '/home/jiang784/Benchmark/DL_Benchmark/models/onnx/squeezenet1.1-7.onnx'

In [34]:
sym, arg_params, aux_params = import_model(model_path)


In [35]:
# Define evaluation metrics
acc_top1 = mx.metric.Accuracy()
acc_top5 = mx.metric.TopKAccuracy(5)

In [36]:
# Define image transforms
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize
])

# Load and process input
val_data = gluon.data.DataLoader(
    imagenet.classification.ImageNet(data_dir, train=False).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers=num_workers)


In [37]:
# Load module
mod = mx.mod.Module(symbol=sym, context=ctx, label_names=None)
mod.bind(for_training=False, data_shapes=[('data', (1,3,224,224))], 
         label_shapes=mod._label_shapes)
mod.set_params(arg_params, aux_params, allow_missing=True)

[22:57:55] ../src/executor/graph_executor.cc:1991: Subgraph backend MKLDNN is activated.


In [38]:
# Compute evaluations
Batch = namedtuple('Batch', ['data'])
acc_top1.reset()
acc_top5.reset()
num_batches = int(50000/batch_size)
print('[0 / %d] batches done'%(num_batches))
# Loop over batches
for i, batch in enumerate(val_data):
    # Load batch
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
    # Perform forward pass
    mod.forward(Batch([data[0]]))
    outputs=mod.get_outputs()
    # Update accuracy metrics
    acc_top1.update(label, outputs)
    acc_top5.update(label, outputs)
    if (i+1)%50==0:
        print('[%d / %d] batches done'%(i+1,num_batches))

[0 / 390] batches done


KeyboardInterrupt: 

In [39]:
# Print results
_, top1 = acc_top1.get()
_, top5 = acc_top5.get()
print("Top-1 accuracy: {}, Top-5 accuracy: {}".format(top1, top5))

Top-1 accuracy: 0.6066706730769231, Top-5 accuracy: 0.8170072115384616


In [27]:
for i, batch in enumerate(val_data):
    # Load batch
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0)
    print(data)
    print(label)
    break

[
[[[[ 1.1529241   1.1529241   1.1357993  ...  1.2042983   1.1529241
     1.0673003 ]
   [ 1.2042983   1.1529241   1.1700488  ...  1.2385478   1.2385478
     1.2385478 ]
   [ 1.2042983   1.1700488   1.1700488  ...  1.2385478   1.2042983
     1.1700488 ]
   ...
   [ 0.878928    0.96455175  1.015926   ...  0.9988013   1.1357993
     1.1700488 ]
   [ 1.1015497   0.96455175  1.1357993  ...  0.9816765   0.878928
     1.1186745 ]
   [ 1.0501755   0.9988013   1.0844251  ...  1.0673003   1.0501755
     1.0501755 ]]

  [[ 1.4306723   1.4831933   1.4306723  ...  1.3956583   1.3781513
     1.3431373 ]
   [ 1.4481792   1.4481792   1.4656863  ...  1.5182073   1.5182073
     1.5707283 ]
   [ 1.4306723   1.4306723   1.4831933  ...  1.5532213   1.5007002
     1.5007002 ]
   ...
   [ 1.0455183   1.1505603   1.2380953  ...  1.2380953   1.3956583
     1.4306723 ]
   [ 1.3081232   1.2030813   1.4131653  ...  1.1855743   1.0805323
     1.3256303 ]
   [ 1.2556022   1.2205883   1.2906163  ...  1.2556022   1.


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
<NDArray 128 @cpu(0)>